In [ ]:
"""
Example of using the SigKit library to create a signal, apply an impairment,
calculate metrics and display visuals.
"""
import numpy as np

from sigkit.core.base import Signal
from sigkit.impairments.awgn import AWGN
from sigkit.metrics.integrity import estimate_snr, calculate_ber

In [ ]:
""" Create a Modulated Signal from Bits """
bits = np.random.randint(0, 2, size=4096)
# TODO: Modulate the bits to create a signal

signal = Signal(
    samples=bits,
    sample_rate=1e6,        # Sample rate in Hz
    carrier_frequency=0.0,  # Carrier frequency in Hz
)

In [ ]:
""" Apply an Impairment """
impairment = AWGN(snr_db=20)  # Target Signal to Noise Ratio in dB
signal: Signal = impairment.apply(signal) # Impairments return a new Signal object for immutability

In [ ]:
""" Calculate Metrics """
ber = calculate_ber(signal.samples, bits)  # Bit Error Rate
snr = estimate_snr(signal.samples, bits)  # Signal to Noise Ratio

In [ ]:
""" Display Visuals """
# TODO